### Importing Required Libraries

In [0]:
import requests
import pandas as pd
from yahoo_fin import stock_info
from pandas_datareader import DataReader
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Retrieving S&P 500 stock information

In [0]:
#Function to retrieve S&P 500 recommendations from Yahoo Finance
def get_recommendations(tickers):
    recommendations = []
    
    for ticker in tickers:
  
        #Yahoo Finance URL for current stock ticker
        url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
        url+= ticker
        url += '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
                'modules=upgradeDowngradeHistory,recommendationTrend,' \
                'financialData,earningsHistory,earningsTrend,industryTrend&' \
                'corsDomain=finance.yahoo.com'
                  
        data = requests.get(url)
        if(not data.ok):
            recommendataion = -1
        
        #Parsing JSON response to retrieve recommendation rating
        try:
          result = data.json()['quoteSummary']['result'][0]
          recommendation = result['financialData']['recommendationMean']['fmt']
        except:
          recommendation = -1

        recommendations.append(recommendation)
    
    return recommendations


#S&P 500 Tickers
tickers = stock_info.tickers_sp500()
recommendations = get_recommendations(tickers)

In [40]:
#Creating a dataframe with stocks and their respective recommendations
columns = ['Company','Recommendations']
data = pd.DataFrame(list(zip(tickers,recommendations)), columns = columns)

#Converting Recommendations column to float
data['Recommendations'] = pd.to_numeric(data['Recommendations'])

data = data[data.Recommendations != -1]

data.sort_values(by=['Recommendations'],)

,Company,Recommendations
387,PWR,1.5
284,LKQ,1.5
72,BSX,1.6
150,DXCM,1.6
255,J,1.6
...,...,...
1,AAL,3.3
154,ED,3.4
61,BEN,3.4
309,MKC,3.5


In [56]:
#Creating Seperate DataFrames to house hold, sell and buy stocka
hold = data[data.Recommendations == 3]
buy = data[data.Recommendations <= 1.5]
sell = data[data.Recommendations >= 4.5]

#Combining theses data frames into one big DF
data_lst = [buy,hold,sell]
new_data = pd.concat(data_lst)
new_data.reset_index(level = 0, inplace = True)

new_data

,index,Company,Recommendations
0,284,LKQ,1.5
1,387,PWR,1.5
2,28,ALLE,3.0
3,97,CLX,3.0
4,119,CTSH,3.0
5,136,DISCK,3.0
6,258,JKHY,3.0
7,267,KIM,3.0
8,274,KSS,3.0
9,276,L,3.0


In [61]:
#Function is used to populate a List with Recommended Stocks from Yahoo Finance
def get_tickers(data):
  tickers = []

  for index, row in data.iterrows():
    tickers.append(row.Company)

  return tickers

#Function is used to fetch news information of stocks recomended by Yahoo Finance
def fetch_news(tickers):
  news = {}

  for ticker in tickers:
    url = 'https://finwiz.com/quote.ashx?t={}'.format(ticker)

    

tickers = get_tickers(new_data)

fetch_news(tickers)

https://finwiz.com/quote.ashx?t=LKQ
https://finwiz.com/quote.ashx?t=PWR
https://finwiz.com/quote.ashx?t=ALLE
https://finwiz.com/quote.ashx?t=CLX
https://finwiz.com/quote.ashx?t=CTSH
https://finwiz.com/quote.ashx?t=DISCK
https://finwiz.com/quote.ashx?t=JKHY
https://finwiz.com/quote.ashx?t=KIM
https://finwiz.com/quote.ashx?t=KSS
https://finwiz.com/quote.ashx?t=L
https://finwiz.com/quote.ashx?t=LEG
https://finwiz.com/quote.ashx?t=ROK
https://finwiz.com/quote.ashx?t=SO
https://finwiz.com/quote.ashx?t=UA
https://finwiz.com/quote.ashx?t=VTR


### Sentiment Analysis